# iTorch / Torch7 implementation


**Gm = d** in lua/torch for LuaJIT. 


## Prerequisites

You need to install `torch` and (if you want to run this notebook) `iTorch`. Works for me on Ubuntu 16.04 and MacOS 10.11. Not gonna say the Mac installation is easy though. Everything was very smooth on Ubuntu.

- [Getting started with Torch](http://torch.ch/docs/getting-started.html)
- [iTorch Requirements](https://github.com/facebook/iTorch#requirements)
- [Torch cheatsheet](https://github.com/torch/torch7/wiki/Cheatsheet)

In [1]:
Plot = require 'itorch.Plot'

In [2]:
-- Implements MATLAB's convmtx.
function convmtx (h, n)
    local m = h:size()[1] - 1
    local c = torch.Tensor(n, n+m):fill(0)
    for i = 1, n do
        c[{ i, {i, i+m} }] = h
    end
    return c
end

In [3]:
convmtx(torch.Tensor({1, -1}), 5)

 1 -1  0  0  0  0
 0  1 -1  0  0  0
 0  0  1 -1  0  0
 0  0  0  1 -1  0
 0  0  0  0  1 -1
[torch.DoubleTensor of size 5x6]



In [4]:
-- Implements bruges's ricker.
function ricker (f, length, dt)
    -- Defaults.
    f = f or 25
    length = length or 0.128
    dt = dt or 0.001

    -- Time basis.
    local t = torch.range(0, length, dt) - length/2
    
    -- Amplitudes.
    local x = 1 - 2 * math.pi * f^2 * torch.pow(t, 2)
    local y = torch.exp(-math.pi^2 * f^2 * torch.pow(t, 2))
    local a = torch.cmul(x, y)

    return t, a
end

## Construct the model

In [5]:
--                         VP    RHO
imp = torch.ones(51, 1) * 2550 * 2650
imp[{{11, 15}, {}}] =     2700 * 2750   
imp[{{16, 27}, {}}] =     2400 * 2450
imp[{{28, 35}, {}}] =     2800 * 3000

In [6]:
x = torch.range(1, imp:size(1))
plot = Plot():line(x, imp[{{},1}])
             :title('Impedance')
             :draw()

In [7]:
numerator = imp[{{2, imp:size(1)}}] - imp[{{1, imp:size(1) - 1}}]
denominator = imp[{{2, imp:size(1)}}] + imp[{{1, imp:size(1) - 1}}]
m = torch.cdiv(numerator, denominator)

In [8]:
x = torch.range(1, m:size(1))
plot = Plot():line(x, m[{{}, 1}], 'blue')
             :title('Reflectivity')
             :draw()

## Forward operator: convolution with wavelet

Now we make the kernel matrix *G*, which represents convolution.

In [9]:
t, wavelet = ricker(100, 0.02, 0.001)

In [10]:
plot = Plot():line(t, wavelet)
             :title('Wavelet')
             :draw()

In [11]:
G_ = convmtx(wavelet, m:size(1))[{{}, {11,60}}]

Right now, there are 50 data samples and 50 model samples. We'd like to make the problem underdetermined. So we only want the odd rows. I can't find anything quite as elegant as Python's slice 'step', but we can easily make `range` with only odd numbers using a step size, then use it to index the temporary `G_` matrix I just made: 

In [12]:
odd_rows = torch.range(1, G_:size(1), 2):long()
G = G_:index(1, odd_rows)

In [13]:
d = G * m

In [14]:
x = torch.range(1, d:size(1))
plot = Plot():circle(x, d[{{}, 1}])
             :title('Data d')
             :draw()

## Noise free: minimum norm

In [15]:
m_est = G:t() * torch.inverse(G * G:t()) * d
d_pred = G * m_est

In [16]:
x = torch.range(1, m_est:size(1))
plot = Plot():line(x, m_est[{{}, 1}])
             :title('Predicted model m_est')
             :draw()

In [17]:
x = torch.range(1, d_pred:size(1))
plot = Plot():circle(x, d_pred[{{}, 1}])
             :title('Predicted data d_pred')
             :draw()

## Least squares solution using GESV

This uses LAPACK. It proceeds via factorization instead of the inverse of (G * G.T). It's the equivalent of calling NumPy or SciPy's `linalg.solve` function.

In [18]:
-- Use torch.gels()
m_est = torch.gels(d, G)
d_pred = G * m_est

-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():circle(x, d[{{},1}], 'blue')
             :circle(x, d_pred[{{},1}], 'red')
             :title('blue:d   red:d_pred')
             :draw()

In [19]:
x = torch.range(1, m_est:size(1))
plot = Plot():line(x, m_est[{{}, 1}])
             :title('Predicted model m_est')
             :draw()

In [20]:
d:dist(d_pred)

9.978417090315e-17	


This is the L2 norm; Mauricio used the square of this as the misfit: 

In [21]:
(d_pred - d):t() * (d_pred - d)

1e-33 *
  9.9569
[torch.DoubleTensor of size 1x1]



## QR factorization

Again, avoiding finding inverses. Uses LAPACK.

As with any QR factorization, `G` must be square.

In [ ]:
q, r = torch.qr(G)
p = q:t() * d
m_est = torch.inverse(r) * p
d_pred = G * m_est

-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():circle(x, d[{{},1}], 'blue')
             :circle(x, d_pred[{{},1}], 'red')
             :title('blue:d   red:d_pred')
             :draw()

In [ ]:
d:dist(d_pred)